In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard
from datetime import datetime
from typing import Final

import numpy as np
from sklearn.datasets import fetch_california_housing
from torch.utils.tensorboard.writer import SummaryWriter

from tabrel.benchmark.nw_regr import Mlp, MlpConfig, generate_multidim_noisy_data, make_random_r, train_nw_arbitrary, NwModelConfig, RelNwRegr

In [ ]:
n_samples: Final[int] = 300
seed: Final[int] = 42
x_dim: Final[int] = 7
n_epochs: Final[int] = 5000

# mse, r2, _, _ = train_nw_arbitrary(
#     x_backgnd=x_np[back_ids],
#     y_backgnd=y_np[back_ids],
#     x_query=x_np[query_ids],
#     y_query=y_np[query_ids],
#     x_val=x_np[val_ids],
#     y_val=y_np[val_ids],
#     r_query_backgnd=r[query_ids][:, back_ids],
#     r_val_nonval=r[val_ids][:, train_ids],
#     cfg=NwModelConfig(input_dim=x_dim, trainable_weights_matrix=True,),
#     lr=1e-3,
#     n_epochs=n_epochs
# )

# print(mse, r2)

In [ ]:
from tabrel.benchmark.nw_regr import train_nw_mlp

def train_nw_mlp_synthetic(
        seed: int,
        mlp_hid_dim: int,
        mlp_out_dim: int,
        dropout: float,
        weight_decay: float,
        writer: SummaryWriter | None,
        trainable_weights: bool = False,
        _n_epochs: int = n_epochs,
        ) -> tuple[float, float]:
    x_np, y_np, c = generate_multidim_noisy_data(n_samples, n_clusters=3, x_dim=x_dim, seed=seed)
    r = make_random_r(seed, c)

    n_query = n_back = n_samples // 3
    n_train = n_query + n_back
    n_val = n_samples - n_train
    back_ids = np.arange(n_back)
    query_ids = np.arange(n_query) + n_back
    val_ids = np.arange(n_val) + n_train

    return train_nw_mlp(
        x=x_np,
        y=y_np,
        r=r,
        back_ids=back_ids,
        query_ids=query_ids,
        val_ids=val_ids,
        mlp_hid_dim=mlp_hid_dim,
        mlp_out_dim=mlp_out_dim,
        dropout=dropout,
        weight_decay=weight_decay,
        _n_epochs=_n_epochs,
        writer=writer,
        trainable_weights=trainable_weights,
        seed=seed,
    )

  

In [ ]:
log_dir: Final[str] = "tb_logs" + datetime.isoformat(datetime.now()).replace(":", "_")
print(log_dir)

In [ ]:
# %tensorboard --logdir {log_dir}

# writer = SummaryWriter(log_dir=log_dir)
# train_nw_mlp_synthetic(
#     seed=seed,
#     mlp_hid_dim=64,
#     mlp_out_dim=6,
#     dropout=.2,
#     weight_decay=1e-4,
#     writer=writer)

mses, r2s = [], []
for _seed in range(15):
    # mse, r2 = train_nw_mlp_synthetic(
    #     seed=_seed,
    #     weight_decay=0.0006913422,
    #     mlp_hid_dim=8,
    #     mlp_out_dim=20,
    #     dropout=0.09302061,
    #     trainable_weights=False,
    #     writer=None,
    # )
    # dim 6:
    # 0.23309 var 0.00074
    # 0.81283 var 0.00048
    # dim 7:
    # 0.08264 var 0.00061
    # 0.93438 var 0.00047

    mse, r2 = train_nw_mlp_synthetic(
        seed=_seed,
        weight_decay=0.002,
        mlp_hid_dim=10,
        mlp_out_dim=20,
        dropout=.19,
        _n_epochs=n_epochs,
        trainable_weights=False,
        writer=None,
    )
    # dim 6:
    # 0.23555 var 0.00118
    # 0.81012 var 0.00048

    # dim 7:
    # 0.07144 var 0.00020
    # 0.94344 var 0.00017
    mses.append(mse)
    r2s.append(r2)
    



for metrics in (mses, r2s):
    print(f"{np.mean(metrics):.5f} var {np.var(metrics):.5f}")

In [ ]:
import optuna

def objective(trial: optuna.Trial) -> float:
    weight_decay = trial.suggest_float("weight_decay", 0., 1e-2)
    mlp_hid_dim = trial.suggest_int("mlp_hid_dim", 6, 100)
    mlp_out_dim = trial.suggest_int("mlp_out_dim", 1, 20)
    dropout = trial.suggest_float("dropout", 0., .5)
    val_r2 = train_nw_mlp_synthetic(
        seed=seed,
        mlp_hid_dim=mlp_hid_dim,
        mlp_out_dim=mlp_out_dim,
        dropout=dropout,
        weight_decay=weight_decay,
        writer=None,
    )[1]
    return val_r2

study = optuna.create_study(direction="maximize", study_name=f"r_random_{datetime.now()}", storage="sqlite:///db.sqlite3")
study.optimize(objective)

code for loading an existing trial

In [ ]:
study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///db.sqlite3",
    study_name="r_deterministic_2025-11-07 14:52:56.356900",
    load_if_exists=True,
)

# study.optimize(objective, n_trials=10)
print(f"Best trial: {study.best_trial}")


In [ ]:
study.best_params

In [ ]:
%tensorboard --logdir {log_dir} 

writer = SummaryWriter(log_dir=log_dir)
train_nw_mlp_synthetic(
    seed=seed,
    mlp_hid_dim=study.best_params["mlp_hid_dim"],
    mlp_out_dim=study.best_params["mlp_out_dim"],
    dropout=study.best_params["dropout"],
    weight_decay=study.best_params["weight_decay"],
    writer=writer,
    _n_epochs = 10_000
)

In [ ]:
# TODO transform Y (with another MLP?)
# TODO embeddings AND learnable norm (+ weight decay?)
# TODO try RTDL num embeddings instead of or before perceptrons
# TODO another synthetic?